In [25]:
import pandas as pd
from thefuzz import process

In [19]:
archivos = ['MercaWeek_Carulla.xlsx', 'MercaWeek_Exito.xlsx'
            , 'Cumpleaños_Carulla.xlsx']


for archivo in archivos:
    df = pd.read_excel(archivo)
    
    if 'Precio regular' in df.columns and 'Precio promocional' in df.columns:

        df = df[df['Precio regular'] != df['Precio promocional']]
        
        df['% de descuento'] = round((1 - df['Precio promocional'] / df['Precio regular']) * 100)
        df['% de descuento'] = df['% de descuento'].astype(int).astype(str) + '%'
        

        df.to_excel(archivo, index=False)
        print(f'Archivo {archivo} actualizado correctamente con la columna % de descuento.')
    else:
        print(f'El archivo {archivo} no contiene las columnas necesarias.')
    

Archivo MercaWeek_Carulla.xlsx actualizado correctamente con la columna % de descuento.
Archivo MercaWeek_Exito.xlsx actualizado correctamente con la columna % de descuento.
Archivo Cumpleaños_Carulla.xlsx actualizado correctamente con la columna % de descuento.


In [38]:
df_comparables = pd.read_excel('comparables.xlsx')
df_comparables.head()

,LLAVE,EAN,Cod. Article,Cod. Article.1,Gramaje,UM,Artículos encontrados comparables Éxito Suba,LINK
0,27UN,7.702511e+12,27,ARROZ DIANA BOLSA 500g,500.0,UN,Arroz Blanco Diana X 500 Gr DIANA 500 gr,https://www.exito.com/arroz-diana-500-gr-479512/p
1,27R01,2.000000e+12,27,ARROZ DIANA BOLSA 12500g,12500.0,R01,Arroz Blanco Diana X 500 Gr DIANA 12500 gr,https://www.exito.com/arroz-blanco-arroba-1250...
2,34UN,7.702084e+12,34,HARINA PAN MAIZ BLANCO BOLSA 1000g,1000.0,UN,harina-pan-maiz-blanca-x-1000-gr-,https://www.exito.com/harina-pan-maiz-blanca-x...
3,38UN,7.702032e+12,38,CAFE SELLO ROJO BOLSA 250g,250.0,UN,CAFE MOLIDO SELLO ROJO 250 gr,https://www.exito.com/cafe-molido-34895/p
4,43UN,7.703812e+12,43,SAL REFISAL ALTA PUREZA BOLSA 1000 g,1000.0,UN,sal-refinada-1000-gr-,https://www.exito.com/sal-refinada-1000-gr-345...


In [68]:
df.head()

,Divisón,Categoría,Subcategoría,Nombre,Link,Precio promocional,Precio regular,% de descuento
0,"Lácteos, huevos y refrigerados",Quesos y quesitos,Queso Tajado,Queso Mozarella FINESSE 30 Tajadas(450 gr),https://www.carulla.com/queso-fresco-15-pcto-d...,21225,28300,25%
1,Limpieza del hogar,Detergentes,Líquido,Jabón Líquido COCO VARELA Doypack Coco (1800 ml),https://www.carulla.com/jabon-liquido-doypack-...,26880,38700,31%
2,"Lácteos, huevos y refrigerados",Leche,Leche Deslactosada,Leche Deslactosada ALPINA Bolsa x6und 1100ml (...,https://www.carulla.com/leche-deslactosada-six...,31940,31950,0%
3,"Bebidas, pasabocas y dulces",Bebidas,Gaseosas y Sodas,Gaseosa Coca Cola ZERO Lata x 12 und (2115 ml),https://www.carulla.com/gaseosa-sin-azucar-pg-...,18900,27000,30%
4,Limpieza del hogar,Detergentes,Líquido,Detergente Líquido FAB Ultra Flash Mega Pack (...,https://www.carulla.com/detergente-liquido-ult...,19880,29500,33%


In [77]:
df_comparables['Mejor_Articulo_Exito'] = ''
df_comparables['Puntaje'] = 0
df_comparables['Comparacion'] = ''
df_comparables_final = pd.DataFrame()

umbral = 87

for archivo in archivos:
    df = pd.read_excel(archivo)
    print(archivo, df.shape)
    for i, row in df_comparables.iterrows():
        articulo_ara = row['LINK']
        mejor_match = process.extractOne(articulo_ara, df['Link'])
        
        
        if mejor_match[1] >= umbral:
            index_mejor_match = df[df['Link'] == mejor_match[0]].index[0]
            df_comparables.at[i, 'Link Articulo Comparable'] = df.at[index_mejor_match, 'Link']
            df_comparables.at[i, 'Nombre Articulo Comparable'] = df.at[index_mejor_match, 'Nombre']
            df_comparables.at[i, 'Precio promocional'] = df.at[index_mejor_match, 'Precio promocional']
            df_comparables.at[i, 'Precio regular'] = df.at[index_mejor_match, 'Precio regular']
            df_comparables.at[i, '% de descuento'] = df.at[index_mejor_match, '% de descuento']
            df_comparables.at[i, 'Puntaje'] = mejor_match[1]
            df_comparables.at[i, 'Comparacion'] = 'Comparable'
        else:
            df_comparables.at[i, 'Articulo Comparable'] = 'No comparable'
            df_comparables.at[i, 'Puntaje'] = mejor_match[1]
            df_comparables.at[i, 'Comparacion'] = 'No comparable'
    
    df_comparables.to_excel(archivo[:-5] + 'Comparables.xlsx', index=False)


    
    df_comparables_final = pd.concat([df_comparables_final, df_comparables[df_comparables['Comparacion'] != 'No comparable']], ignore_index=True)
    
df_comparables_final.drop(['Puntaje','Comparacion'], axis=1, inplace = True)

df_comparables_final.to_excel('Todos_Comparables.xlsx', index=False)
#df_comparables.head()

MercaWeek_Carulla.xlsx (218, 8)
MercaWeek_Exito.xlsx (335, 8)
Cumpleaños_Carulla.xlsx (217, 8)


In [75]:
df_comparables_final

,LLAVE,EAN,Cod. Article,Cod. Article.1,Gramaje,UM,Artículos encontrados comparables Éxito Suba,LINK,Mejor_Articulo_Exito,Articulo Comparable,Link Articulo Comparable,Nombre Articulo Comparable,Precio promocional,Precio regular,% de descuento
0,3002UN,7.702535e+12,3002,BEBIDA GASEOSA COCA-COLA S/AZ PET 400ml,400.0,UN,Gaseosa Sin Azúcar COCA COLA 400 ml,https://www.exito.com/bebida-gaseosa-original-...,,NaN,https://www.carulla.com/bebida-gaseosa-origina...,Gaseosa Coca Cola ZERO Botella personal (400 ml),2240.0,3200.0,30%
1,3308UN,7.702535e+12,3308,JUGO DEL VALLE FRESH NARANJA 2500ml X 2,5000.0,UN,Jugo Fresh Frutas Cítricas x 2 Unds DEL VALLE ...,https://www.exito.com/bebida-naranja-citrus-bi...,,NaN,https://www.carulla.com/bebida-naranja-citrus-...,Jugo DEL VALLE Fresh frutas cítricas (5000 ml),8960.0,12800.0,30%
2,3611UN,7.702535e+12,3611,DUO PACK COCACOLA + QUATRO C/U X 3000 ml,6000.0,UN,GASEOSA COCA COLA SABOR ORIGINAL + QUATRO 3000 ML,https://www.exito.com/duopack-coca-cola-quatro...,,NaN,https://www.carulla.com/duopack-coca-cola-quat...,Gaseosa COCA COLA Original + Quatro x 2und 3L ...,11775.0,15700.0,25%
3,3643UN,7.702032e+12,3643,CAFE COLCAFE CLASICO VIDRIO 170g,170.0,UN,Cafe instantaneo clasico COLCAFE 170 gr,https://www.exito.com/cafe-instantaneo-x-170-g...,,NaN,https://www.carulla.com/cafe-instantaneo-x-170...,cafe COLCAFE (170 gr),14850.0,19800.0,25%
4,4891UN,7.702609e+12,4891,BEBIDA GASEOSA KOLA ROMAN X 1500ml,1500.0,UN,Gaseosa Sabor Original KOLA ROMAN 1500 ml,https://www.exito.com/gaseosa-sabor-original-k...,,NaN,https://www.carulla.com/gaseosa-sabor-original...,Gaseosa KOLA ROMAN Original (1500 ml),3080.0,4900.0,37%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,14413UN,7.702535e+12,14413,AGUA BRISA SIN GAS PET 3000ml,3000.0,UN,Agua Sin gas BRISA 3000 ml,https://www.exito.com/agua-sin-gas-pet-brisa-3...,,NaN,https://www.carulla.com/agua-sin-gas-pet-brisa...,Agua BRISA Sin gas (3000 ml),4080.0,5100.0,20%
69,16899UN,7.702535e+12,16899,FOURPACK DEL VALLE SURTIDO 4UN C/U 188ml,752.0,UN,Jugo Frutal Surtido x 6 Unds DEL VALLE 940 ml,https://www.exito.com/of-refresco-surtidos-del...,,NaN,https://www.carulla.com/of-refresco-surtidos-d...,Jugo DEL VALLE Frutal surtido x6und 188ml (940...,5390.0,7700.0,30%
70,18143UN,7.709183e+07,18143,GASEOSA KOLA ROMAN 250 ml,250.0,UN,Gaseosa Sabor Original KOLA ROMAN 400 ml,https://www.exito.com/gaseosa-kola-180610/p,,NaN,https://www.carulla.com/gaseosa-kola-180610/p,Gaseosa KOLA ROMAN Original (400 ml),1820.0,2600.0,30%
71,18569UN,7.702535e+12,18569,DEL VALLE FRESH CITRUS 400ML,400.0,UN,Jugo Fresh Frutas Cítricas DEL VALLE 400 ml,https://www.exito.com/bebida-naranja-citrus-de...,,NaN,https://www.carulla.com/bebida-naranja-citrus-...,Jugo DEL VALLE Fresh frutas cítricas (400 ml),1400.0,2000.0,30%


In [ ]:
df_comparables_final.shape()